# Random forest support vector regressor (RF-SVR)
## Vincent Buekers
Promotor: Prof. dr. Johan A.K. Suykens

Supervision: Yingyi Chen

## Partition input space into subsets 
Subsets are obtained from the leaf nodes of an extremely randomized tree. For purposes of theoretical consistency, only one candidate feature is selected from all d features using the option max_features = 1, yielding totally random trees.

Note: these are non-overlapping subsets due to the recursive branching mechanism in decision trees

In [2]:
def extra_partition(X_train,X_test, y_train,y_test, idx_train,idx_test):

    subsets = []
    
    # totally randomized tree (max_features=1)
    extra = tree.ExtraTreeRegressor(max_features=1,min_samples_leaf = int(np.sqrt(len(X_train))) )
    extra.fit(X_train,y_train)
    
    # obtain leaf indices the datapoints appear in
    leaf_idx_train, leaf_idx_test = extra.apply(X_train), extra.apply(X_test)
    
    # Keep track of observation indexes and prepare for pandas' .groupby
    leaf_idx_train = pd.DataFrame(leaf_idx_train, index=idx_train)
    leaf_idx_test = pd.DataFrame(leaf_idx_test, index=idx_test)
    
    # Group train and test observations by their leaf node
    groups_train = leaf_idx_train.groupby(leaf_idx_train[0],axis=0).groups
    groups_test = leaf_idx_test.groupby(leaf_idx_test[0],axis=0).groups
    
    # collect all data back into one array, sorted by original observation indexes
    X_train, X_test = np.c_[idx_train,X_train], np.c_[idx_test,X_test]
    y_train, y_test = np.c_[idx_train,y_train], np.c_[idx_test,y_test]
    X, y = np.r_[X_train,X_test], np.r_[y_train,y_test]
    X, y = X[np.argsort(X[:,0])], y[np.argsort(y[:,0])]
    X, y = np.delete(X, 0, 1), np.delete(y, 0, 1)
    
    # tree predictions (only test observations will be retrieved later on)
    preds_tree = extra.predict(X).reshape(-1,1)
    
    # Obtain train and test subsets created by the leaf node partitioning
    # iterables are a list of Int64index objects for the data in each leaf node
    for leaf_train, leaf_test in zip(list(groups_train.values()),list(groups_test.values())) :
        
        # subset the data
        X_train_sub, y_train_sub = X[leaf_train], y[leaf_train]
        X_test_sub, y_test_sub, y_tree = X[leaf_test] ,y[leaf_test], preds_tree[leaf_test]
        
        # original indexes of the observations appearing in this leaf
        train_indexes = np.array(leaf_train).reshape(-1,1)
        test_indexes = np.array(leaf_test).reshape(-1,1)
        
        # training subset including original observation indexes
        sub_train = np.concatenate((train_indexes, X_train_sub, y_train_sub), axis=1)
        # testing subset including original observation indexes and tree predictions
        sub_test = np.concatenate((test_indexes, X_test_sub, y_test_sub, y_tree), axis=1)

        subsets.append([sub_train,sub_test])
    
    return subsets

# Embedded SVM Regressor
for each subset an svm classifier is trained on the training subset and used to predict the corresponding leaf test test.

Since support vector machines are not scale-invariant, the data are scaled to have zero mean and unit variance for each local SVM 

In [11]:
def fit_svr_linear(subset):
    
    idx_train, X_train, y_train = subset[0][:,0], subset[0][:,1:-1], subset[0][:,-1]
    idx_test,X_test,y_test = subset[1][:,0],subset[1][:,1:-1],subset[1][:,-1]
        
    # decide whether to solve in primal or dual
    QP_bool = False if (X_train.shape[0] > X_train.shape[0]) else True
        
    # fit svm to subset
    reg = svm.LinearSVR(dual=QP_bool)
    reg.fit(X_train,y_train)
        
    # obtain predictions for subset
    svm_pred = reg.predict(X_test)
    svm_pred = np.concatenate((idx_test.reshape(-1,1), svm_pred.reshape(-1,1)), axis=1)
        
    return svm_pred

In [11]:
def fit_svr_sgd(subset):
    
    idx_train, X_train, y_train = subset[0][:,0], subset[0][:,1:-1], subset[0][:,-1]
    idx_test,X_test,y_test = subset[1][:,0],subset[1][:,1:-1],subset[1][:,-1]
                
    # fit svm to subset
    reg = linear_model.SGDRegressor(early_stopping=True)
    reg.fit(X_train,y_train)
        
    # obtain predictions for subset
    svm_pred = reg.predict(X_test)
    svm_pred = np.concatenate((idx_test.reshape(-1,1), svm_pred.reshape(-1,1)), axis=1)
        
    return svm_pred

In [11]:
def fit_svr_kernel(subset):
    
    idx_train, X_train, y_train = subset[0][:,0], subset[0][:,1:-1], subset[0][:,-1]
    idx_test,X_test,y_test = subset[1][:,0],subset[1][:,1:-1],subset[1][:,-1]
                
    # fit svm to subset
    reg = svm.SVR()
    reg.fit(X_train,y_train)
        
    # obtain predictions for subset
    svm_pred = reg.predict(X_test)
    svm_pred = np.concatenate((idx_test.reshape(-1,1), svm_pred.reshape(-1,1)), axis=1)
        
    return svm_pred

# concurrent SVM execution across leaf nodes

In [4]:
def svr_tree_linear(X_train,X_test, y_train,y_test, idx_train,idx_test):
    
    # Obtain subsets
    subsets = get_groups(X_train,X_test, y_train,y_test, idx_train,idx_test)
    
    preds_leaf = []
    
    # Run SVM's in parallel
    with Parallel() as parallel:
        result = parallel(delayed(fit_svr_linear)(subset) for subset in subsets)
        preds_leaf.append(result)
        
    # aggregate predictions of the leafs into one set of predictions for the tree
    preds_all = np.concatenate(preds_leaf[0],axis=0)
    
    # sort predictions by their index
    preds_sorted = preds_all[np.argsort(preds_all[:,0])]
    
    return preds_sorted[:,1].reshape(-1,1) # return predictions without index

In [4]:
def svr_tree_sgd(X_train,X_test, y_train,y_test, idx_train,idx_test):
    
    # Obtain subsets
    subsets = tree_partition(X_train,X_test, y_train,y_test, idx_train,idx_test)
    
    preds_leaf = []
    
    # Run SVM's in parallel
    with Parallel() as parallel:
        result = parallel(delayed(fit_svr_sgd)(subset) for subset in subsets)
        preds_leaf.append(result)
        
    # aggregate predictions of the leafs into one set of predictions for the tree
    preds_all = np.concatenate(preds_leaf[0],axis=0)
    
    # sort predictions by their index
    preds_sorted = preds_all[np.argsort(preds_all[:,0])]
    
    return preds_sorted[:,1].reshape(-1,1) # return predictions without index

In [4]:
def svr_tree_kernel(X,y, X_train,X_test, y_train,y_test, idx_train,idx_test):
    
    # Obtain subsets
    subsets = tree_partition(X_train,X_test, y_train,y_test, idx_train,idx_test)
    
    preds_leaf = []
    
    # Run SVM's in parallel
    with Parallel() as parallel:
        result = parallel(delayed(fit_svr_kernel)(subset) for subset in subsets)
        preds_leaf.append(result)
        
    # aggregate predictions of the leafs into one set of predictions for the tree
    preds_all = np.concatenate(preds_leaf[0],axis=0)
    
    # sort predictions by their index
    preds_sorted = preds_all[np.argsort(preds_all[:,0])]
    
    return preds_sorted[:,1].reshape(-1,1) # return predictions without index

## Run the above procedure for multiple trees in parallel
This results in a support vector machine embedded random forest

In [5]:
# Run trees in parallel using all cores
def rf_svr(X,y):
    
    # amount of trees in forest
    trees = 100
    
    forest_pred=[]
    
    X_train,X_test, y_train,y_test, idx_train,idx_test = train_test_split(X,y
                                                                          ,np.arange(len(X))
                                                                          ,test_size=1/3)
    t = time.time()
    
    with Parallel() as parallel:
        forest_pred = parallel(delayed(svm_parallel)(X,y 
                                                     ,X_train,X_test 
                                                     ,y_train,y_test
                                                     ,idx_train,idx_test) for i in range(1,trees))
    
    # training time
    training_time = time.time() - t
    
    # reshape array such that column k denotes prediction for tree k
    forest_pred = np.concatenate(forest_pred,axis=1)
    
    mean_pred = np.apply_along_axis(np.mean, 1, forest_pred) 
    
    test = y_test[np.argsort(idx_test)]
    
    R_2 = metrics.r2_score(test, mean_pred)
    
    return R_2, training_time